In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import os

from neyman.inferences import batch_hessian
import itertools as it
from tqdm import tnrange
import corner


repo_path = ".."
import sys
sys.path.append(f"{repo_path}/code/")

from synthetic_3D_example import SyntheticThreeDimExample
from template_model import TemplateModel
import tensorflow_probability as tfp

from summary_statistic_computer import SummaryStatisticComputer

ds = tfp.distributions
ge = tf.contrib.graph_editor
k = tf.keras

font = {'size'   : 14}

matplotlib.rc('font', **font)
figure = {'figsize'   : (12,8),
          'max_open_warning': False}
matplotlib.rc('figure', **figure)

## Problem description

The considered distributions is a mixture of two 3D distributions, referred as $s(\textbf{x})$
and $b(\textbf{x})$. 

The background distribution $s(\textbf{x})$ consist of a 2D gaussian in the
fist two dimensions centred at $(r,0.)$ where r is a nuissance parameter loosely constrained
around $r=2.$. The last dimension is a exponential distribution with a rate $\lambda$ loosely
constrained around $\lambda=3$ which will be the second nuissance parameter.

The  signal distribution $s(\textbf{x})$ consist of a 2D gaussian in the
fist two dimensions centred at $(0.,0.)$.
The last dimension is a exponential distribution with a rate of $2.0$.

Analytically the mixture density considered can be expressed as:
$$ m(\textbf{x}) = \mu \cdot s(\textbf{x}) + (1-\mu) \cdot b(\textbf{x}) $$
However, a different parametetrization will be considered, in which our parameter of interest
will be denoted as $s_{\textrm{exp}}$ corresponding to the expected number of signal observations, which we expect to be around 100.
The expected number of background observations will be the third nuissance parameter, which will
be loosely constrained around $b_{\textrm{exp}} = 1000$. The total number of
expected observations will be distributed as $\textrm{Poisson}(s_{\textrm{exp}}+b_{\textrm{exp}}$). The mixture fraction $\mu$ can be expressed in this parametrization as:
$$ \mu = \frac{s_{\textrm{exp}}}{s_{\textrm{exp}}+b_{\textrm{exp}}}$$

In [ ]:
SyntheticThreeDimExample.__init__??

In [ ]:
SyntheticThreeDimExample.transform_bkg??

In [ ]:
problem = SyntheticThreeDimExample()

x_values = tf.placeholder(dtype=tf.float32, shape=(None, 3), name="x_values")

phs = { problem.r_dist : 2.,
        problem.b_rate : 3.,
        problem.b_exp : 1000,
        problem.s_exp : 50}

In [ ]:

n_s = 50000

s_sam = problem.s_dist.sample(n_s, seed=27)
b_sam = problem.b_dist.sample(n_s, seed=27)
with tf.Session() as sess:
  b_sample = sess.run(b_sam, phs)
  s_sample = sess.run(s_sam, phs)

labels = [r"$x_0$", "$x_1$", "$x_2$"]
ran = [(-10,10),
       (-10,10),
       (0,4)]
bins = [np.linspace(-10,10,40, endpoint=True),
        np.linspace(-10,10,40, endpoint=True),
        np.linspace(0,4,20, endpoint=True)]

# 2D levels to use
levels = 1.0 - np.exp(-0.5 * np.array([1.,2.,3.]) ** 2)

fig = corner.corner(b_sample, bins=bins,range=ran, color="blue",weights=np.ones(n_s),
                    smooth=0.95, labels=labels,levels=levels)
fig = corner.corner(s_sample, bins=bins,range=ran, color="red",weights=np.ones(n_s),
                    smooth=0.95, labels=labels,levels=levels,fig=fig)

# set 1D hist y-scales
scales = [12000, 12000, 25000]
n_dim = 3
axes = np.array(fig.axes).reshape((n_dim, n_dim))
for i in range(n_dim):
  ax = axes[i, i]
  ax.set_ylim([0,scales[i]])

#fig.savefig("../paper/gfx/figure2a.pdf",bbox_inches="tight")
fig

In [ ]:

n_s = 50000

with tf.Session() as sess:
  weights=np.ones(s_sample.shape[0])
  m_sample = sess.run(problem.m_dist.sample(n_s, seed=17), phs)

labels = [r"$x_0$", "$x_1$", "$x_2$"]
ran = [(-10,10),
       (-10,10),
       (0,4)]
bins = [np.linspace(-10,10,40, endpoint=True),
        np.linspace(-10,10,40, endpoint=True),
        np.linspace(0,4,20, endpoint=True)]

# 2D levels to use
levels = 1.0 - np.exp(-0.5 * np.array([1.,2.,3.]) ** 2)

fig = corner.corner(m_sample, bins=bins, range=ran,
                    levels=levels,color="black",weights=np.ones(n_s),
                    smooth=0.9999, labels=labels)

# set 1D hist y-scales
scales = [12000, 12000, 25000]
n_dim = 3
axes = np.array(fig.axes).reshape((n_dim, n_dim))
for i in range(n_dim):
  ax = axes[i, i]
  ax.set_ylim([0,scales[i]])

#fig.savefig("../paper/gfx/figure2b.pdf",bbox_inches="tight")
fig

In [ ]:
with tf.Session() as sess:
  train_arrays = sess.run(problem.train_data())
  valid_arrays = sess.run(problem.valid_data())   
  test_arrays =  sess.run(problem.test_data())   

## Classifier training

In [ ]:
from synthetic_3D_cross_entropy import SyntheticThreeDimCrossEntropy

seed = 17

n_epochs = 100
lr = 1e-2
batch_size = 
xe_path = f"xe_lr_{lr}_n_e_{n_epochs}_seed_{seed}"

xe_model = SyntheticThreeDimCrossEntropy(model_path=xe_path,
                                         seed=seed)

xe_model.fit(n_epochs=n_epochs, lr=lr,batch_size=batch_size, seed=seed)

In [ ]:
ssc = SummaryStatisticComputer()

In [ ]:
clf_shapes = ssc.classifier_shapes(xe_path)

In [ ]:
def shape_variation_plot(shape_dict):
  fig, ax = plt.subplots(2,1, figsize=(8,8),sharex=True)
  fig.subplots_adjust(hspace = 0.05)
  
  
  bins = np.linspace(0,1,11,endpoint=True)
  centers =   (bins[1:]+bins[:-1])/2.
  width = (bins[1:]-bins[:-1])
  
  plot_options = {("sig",) : (r"signal","red"),
                 ("bkg",2.,3.) : (r"background","blue")}
                 
  for k, option in plot_options.items():
    clf_shapes_norm = shape_dict[k]/shape_dict[k].sum()
    ax[0].bar(x=centers, height=clf_shapes_norm,
              width=width,color=option[1], alpha=0.35,
              label=option[0])
  
  exp_bkg =  (shape_dict[("bkg",2.,3.)]/shape_dict[("bkg",2.,3.)].sum())*1000.
  exp_sig = (shape_dict[("sig",)]/shape_dict[("sig",)].sum())*50.
  sig_art = ax[1].errorbar(x=centers, y=exp_sig/exp_bkg,
                           xerr=width/2.,fmt=".",color="red", label="signal")
  
  plot_options = {("bkg",2.0,2.5) : (r"v","green"),
                  ("bkg",2.0,3.5) : (r"^","green"),
                  ("bkg",1.8,3.0) : (r"v","purple"),
                  ("bkg",2.2,3.0) : (r"^","purple")}
  
  var_lines = []
  var_labels = []
  for k, option in plot_options.items():
    exp_shift_bkg = (shape_dict[k]/shape_dict[k].sum())*1000.
    label = f"bkg $r={k[1]}$ $\\lambda={k[2]}$"
    var_lines.append(ax[1].errorbar(x=centers, y=(exp_shift_bkg-exp_bkg)/exp_bkg,
                     xerr=width/2.,fmt=".", color=option[1],
                     alpha=0.55, marker=option[0]))
    var_labels.append(label)
  
  ax[0].legend(loc="upper center", frameon=False)
  ax[0].set_ylim(top=0.7)
  ax[0].set_ylabel("counts (normalised)")
  
  leg_0 = ax[1].legend(var_lines,var_labels,loc="lower center", ncol=2,frameon=False)
  leg_1 = ax[1].legend([sig_art],["signal"],loc="upper center", frameon=False)

  ax[1].set_ylim([-0.75,0.75])
  ax[1].set_xlim([0.,1.])
  
  ax[1].add_artist(leg_0)
  ax[1].set_ylabel("relative variation")
  ax[1].yaxis.set_major_locator(matplotlib.ticker.MultipleLocator(0.2))
  
  fig.align_ylabels()

  
  return ax, fig
 
axs, fig = shape_variation_plot(clf_shapes)
axs[1].set_xlabel("neural network classifier output")
#fig.savefig("../paper/gfx/figure3a.pdf",bbox_inches="tight")
fig

## Analytical likelihood ratio

Probabilistic classifiers trained to differenciate signal and background observation can be used to approximate the likelihood ratio. In this
particular case, we can easily approximate the likelihood ratio analytically.

$$s(\textbf{x}) = \frac{s(\textbf{x})}{s(\textbf{x}) + b(\textbf{x})}$$

where $s$ is fully specified but $b$ depends on $r$ and $\lambda$, which for
a probabilistic classifier are typically taken as fixed at certain values.

In [ ]:
opt_shapes = ssc.optimal_shapes()

In [ ]:
axs, fig = shape_variation_plot(opt_shapes)

axs[1].set_xlabel("optimal classifier output")
#fig.savefig("../paper/gfx/figure3b.pdf",bbox_inches="tight")

## INFERNO training

In [ ]:
from synthetic_3D_inferno import SyntheticThreeDimInferno

n_epochs = 50
lr = 1e-6
batch_size = 1000
t_train = 0.1
t_eval = 0.05

n_inits = 100
seed = 7


pars = ["s_exp", "r_dist", "b_rate"]

aux = {"r_dist": ds.Normal(loc=2.0, scale=0.4),
       "b_rate": ds.Normal(loc=3.0, scale=1.)}


inf_path = f"inf_ne_{n_epochs}_lr_{lr}_bs_{batch_size}_t_{t_train}"

inferno = SyntheticThreeDimInferno(model_path=inf_path, poi="s_exp",
                                    pars=pars, seed=seed, aux=aux)
inferno.fit(n_epochs=n_epochs, lr=lr, batch_size=batch_size,
            temperature=t_train, seed=seed)

inf_fisher, inf_aux_fisher = inferno.eval_hessian(temperature=t_eval)

In [ ]:
inf_fisher.marginals(pars[0:3])

In [ ]:
inf_shapes = ssc.inferno_shapes(inf_path)

In [ ]:
axs, fig = shape_variation_plot(inf_shapes)

axs[1].set_xlabel("inferno output")
#test

## Analytical Inference

In [ ]:
from extended_model import ExtendedModel

problem = SyntheticThreeDimExample()


In [ ]:
x_values = tf.placeholder(dtype=tf.float32, shape=(None, 3), name="x_values")

aux = {"r_dist" : ds.Normal(loc=2.0, scale=0.4),
       "b_rate" : ds.Normal(loc=3.0, scale=1.0),
       "b_exp" : ds.Normal(loc=1000.0, scale=100.)}
em = ExtendedModel(problem, aux=aux)

In [ ]:
bkg_t = problem.transform_bkg(x_values)

with tf.Session() as sess:
  valid_arrays = sess.run(problem.valid_data())   


In [ ]:
with tf.Session() as sess:
  bkg_t_arr = sess.run(bkg_t, {x_values : valid_arrays["bkg"]})
  obs_phs = {em.s_n_exp : 50.,
               em.b_n_exp : 1000.,
               em.s_data : valid_arrays["sig"],
               em.b_data : bkg_t_arr }
  hess, hess_dict = em.hess(par_phs={},obs_phs=obs_phs)   

In [ ]:
pars = list(em.all_pars.keys())
f = FisherMatrix(hess, pars )
f.marginals(["s_exp"])


In [ ]:
f_total.marginals("")

In [ ]:

f_total = f.add_matrix(sum(hess_dict.values()))
f_total.marginals(pars)

In [ ]:
np.sqrt(np.diag(np.linalg.inv(hess+sum(list(hess_dict.values())))))

In [ ]:
np.sqrt(np.diag(np.linalg.inv(hess[0:1,0:1])))

In [ ]:
with tf.Session() as sess:
  
  r_dist_scan = np.linspace(1.5,2.5,11)
  for r_dist in r_dist_scan:
    par_phs = {problem.r_dist : r_dist}
    bkg_t_arr = sess.run(bkg_t, {x_values : valid_arrays["bkg"],
                               **par_phs})
    obs_phs = {em.s_n_exp : 50.,
               em.b_n_exp : 1000.,
               em.s_data : valid_arrays["sig"],
               em.b_data : bkg_t_arr }
    cov = em.cov(par_phs,obs_phs=obs_phs)
    print(r_dist,np.sqrt(np.diag(cov)))
    grad = sess.run(em.e_grad, {**par_phs,**obs_phs})
    print(cov.dot(grad))